# arXiv Paper Curator -  Infrastructure Setup

Build a production-grade RAG system using Docker, PostgreSQL, OpenSearch, FastAPI, Airflow, and Ollama.

## Technology Stack
| Component | Purpose | Port |
|-----------|---------|------|
| **FastAPI** | REST API | 8000 |
| **PostgreSQL** | Paper metadata storage | 5432 |
| **OpenSearch** | Hybrid search engine | 9200/5601 |
| **Apache Airflow** | Workflow automation | 8080 |
| **Ollama** | Local LLM inference | 11434 |

In [11]:
# Environment Check
import sys
from pathlib import Path

python_version = sys.version_info
print(f"Python Version: {python_version.major}.{python_version.minor}.{python_version.micro}")
print(f"Environment: {sys.executable}")

if python_version >= (3, 12):
    print("✓ Python version compatible")
else:
    print("✗ Need Python 3.12+")
    exit()

Python Version: 3.12.0
Environment: c:\Users\shubh\AppData\Local\Programs\Python\Python312\python.exe
✓ Python version compatible


In [12]:
# Find Project Root
current_dir = Path.cwd()
print(f"Current Directory: {current_dir}")
print(f"Directory Name: {current_dir.name}")
print(f"Parent Directory Name: {current_dir.parent.name}")
print(f"Grandparent Directory Name: {current_dir.parent.parent.name}")

if current_dir.name == "setup" and current_dir.parent.name == "notebooks":
    project_root = current_dir.parent.parent
elif (current_dir / "compose.yml").exists():
    project_root = current_dir
else:
    project_root = None

if project_root and (project_root / "compose.yml").exists():
    print(f"✓ Project root: {project_root}")
else:
    print("✗ Missing compose.yml - check directory")
    exit()
    

Current Directory: c:\Users\shubh\OneDrive\Documents\RAG_Research_Paper_Assistant\notebooks\setup
Directory Name: setup
Parent Directory Name: notebooks
Grandparent Directory Name: RAG_Research_Paper_Assistant
✓ Project root: c:\Users\shubh\OneDrive\Documents\RAG_Research_Paper_Assistant


In [13]:
# Check Docker
import subprocess

try:
    result = subprocess.run(["docker", "--version"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print(f"✓ Docker: {result.stdout}")
    else:
        print("✗ Docker: Not working")
        exit()
except:
    print("✗ Docker: Not found")
    exit()

✓ Docker: Docker version 29.1.3, build f52814d



In [14]:
# Check Docker Compose
try:
    result = subprocess.run(["docker", "compose", "version"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print(f"✓ Docker Compose: {result.stdout.split()[3]}")
    else:
        print("✗ Docker Compose: Not working")
        exit()
except:
    print("✗ Docker Compose: Not found")
    exit()

✓ Docker Compose: v5.0.1


In [15]:
# Check UV Package Manager
try:
    result = subprocess.run(["uv", "--version"], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print(f"✓ UV: {result.stdout.strip()}")
        print("\n✓ All required software ready!")
    else:
        print("✗ UV: Not working")
        exit()
except:
    print("✗ UV: Not found")
    exit()

✓ UV: uv 0.9.28 (0e1351e40 2026-01-29)

✓ All required software ready!


## Start Services

**Command to run (in terminal):**
```bash
cd [project-root]
docker compose up -d
```

**What this does:** Downloads images (first time) and starts all services in background.